This notebook expects a 'standards' CSV file which contains two (in future three) columns in the following order:  

1. Names of the standards (e.g. isotope names) - string
2. The factor by which to divide for quantification (e.g. slope of calibration curve) - float
3. (Not yet supported) The y-axis section of a calibration curve - float

In [ ]:
import io
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from ipywidgets import SelectMultiple, TwoByTwoLayout, interact, widgets

### Enter the working direcotry paths & select your single-cell data and standards files

In [ ]:
singlecell_files = widgets.FileUpload(
    accept=".csv",
    multiple=True,
    description="Upload data csv's",
    layout={"width": "auto"},
)

standard_file = widgets.FileUpload(
    accept=".csv",
    multiple=True,
    description="Upload standards csv",
    layout={"width": "auto"},
)

save_path_widget = widgets.Text(
    value="",
    placeholder="Enter working directory/desired save path",
    description="Path:",
    disabled=False,
)
display(save_path_widget)

display(singlecell_files, standard_file)

In [ ]:
# Reading and formatting the single-cell and standard files
singlecell_dfs = [
    pd.read_csv(io.BytesIO(file.content)) for file in singlecell_files.value
]

# Assuming you upload only one standard file
uploaded_standard_file = standard_file.value[0]
standards_df = pd.read_csv(
    io.BytesIO(uploaded_standard_file.content),
    dtype={"a": str, "b": np.float64, "c": np.float64},
    delimiter=";",
    header=None,
)

standards_df = standards_df.transpose()
standards_df.columns = standards_df.iloc[0]
standards_df = standards_df.drop(0)
standards_df = standards_df.reset_index(drop=True)
standards_df

### Select the desired channels and the corresponding standard isotopes
Make sure you select channel and corresponding isotopes in the same order otherwise you will obtain false results

In [ ]:
# Select the desired channel descriptions + the corresponding standard isotopes
quant_channels = widgets.SelectMultiple(
    options=singlecell_dfs[0].columns,
    description="Channels for quantification",
    disabled=False,
    rows=10,
    style={"description_width": "initial"},
    layout=widgets.Layout(width="auto", height="auto"),
)

std_isotopes = widgets.SelectMultiple(
    options=standards_df.columns,
    description="Standard isotopes for quantification",
    disabled=False,
    rows=10,
    style={"description_width": "initial"},
    layout=widgets.Layout(width="auto", height="auto"),
)

# Initialize lists to hold selected options
quant_channels_selected = []
std_isotopes_selected = []


def on_quant_channels_change(change):
    if change["type"] == "change" and change["name"] == "value":
        # Add any newly selected options to the end of the list
        for elem in change["new"]:
            if elem not in quant_channels_selected:
                quant_channels_selected.append(elem)
        # Remove any options that were deselected
        for elem in quant_channels_selected:
            if elem not in change["new"]:
                quant_channels_selected.remove(elem)


def on_std_isotopes_change(change):
    if change["type"] == "change" and change["name"] == "value":
        # Add any newly selected options to the end of the list
        for elem in change["new"]:
            if elem not in std_isotopes_selected:
                std_isotopes_selected.append(elem)
        # Remove any options that were deselected
        for elem in std_isotopes_selected:
            if elem not in change["new"]:
                std_isotopes_selected.remove(elem)


# Attach change listeners to the widgets
quant_channels.observe(on_quant_channels_change)
std_isotopes.observe(on_std_isotopes_change)

TwoByTwoLayout(top_left=quant_channels, top_right=std_isotopes)

### Plot quantified channels of all datasets

In [ ]:
# Quantifiy the selected isotopes
quant_df = singlecell_dfs.copy()

for df in quant_df:
    for i, channel in enumerate(quant_channels_selected):
        df[channel] = (
            df[channel] / standards_df.iloc[0][std_isotopes_selected[i]]
        )
        
fig, axs = plt.subplots(
    len(quant_channels_selected), len(quant_df), figsize=(16, 9), sharey=False
)


# Helper function to ensure axs is a 2D array
def ensure_2d_axs(axs):
    if axs.ndim == 1:
        if len(quant_channels_selected) == 1 or len(quant_df) == 1:
            return axs.reshape(len(quant_channels_selected), len(quant_df))
    return axs


axs = ensure_2d_axs(axs)

# Extracting the uploaded single cell file names from the tuple of dictionaries
uploaded_filenames = [file_dict["name"] for file_dict in singlecell_files.value]

for i, c in enumerate(quant_channels_selected):
    for j, df in enumerate(quant_df):
        ax = sns.histplot(df, x=c, ax=axs[i, j])
        ax.set(xlabel=f"{c} content in fg/cell")

        if j == 0:
            ax.set(ylabel="Cell Count")

        # Using the extracted file's name for the title
        ax.set_title(f"Quantified {c} distribution of {uploaded_filenames[j][:-4]}")

        fig.tight_layout()

plt.subplots_adjust(hspace=0.3, wspace=0.2)
plt.show()

### Uncomment the cell below to save individual histograms

In [ ]:
# Uncomment the section below to save individual histograms rather than displaying them
for idx, df in enumerate(quant_df):
    for c in quant_channels.value:
        ax = sns.displot(df, x=c)

        file_name = list(singlecell_files.value[idx].values())[0]
        file_base_name = os.path.splitext(file_name)[0]

        plt.title(f"Quantified {c} distribution of {file_base_name}")
        ax.set(xlabel=f"{c} content in fg/cell", ylabel="Cell Count")
        ax.figure.tight_layout()

        save_path = os.path.join(
            save_path_widget.value, f"{file_base_name}_quantified_{c}_hist.png"
        )
        plt.savefig(fname=save_path, dpi=600)
        plt.close()

### Uncomment the cell below to save quantified data as .csv files

In [ ]:
# Uncomment code below to save quantified data as .csv files
for idx, df in enumerate(quant_df):
    # Make a copy of the dataframe to avoid modifying original df
    df_copy = df.copy()

    # Append '_quantified' to column names selected by quant_channels
    rename_dict = {c: c + "_quantified" for c in quant_channels.value}
    df_copy.rename(columns=rename_dict, inplace=True)

    # Generate save path and save the modified dataframe
    file_name = list(singlecell_files.value[idx].values())[0]
    file_base_name = os.path.splitext(file_name)[0]

    save_path = os.path.join(save_path_widget.value, f"{file_base_name}_quantified.csv")
    df_copy.to_csv(save_path, index=False)